In [1]:
import pandas as pd
import numpy as np
import requests
import urllib.request as request
from sklearn.preprocessing import OneHotEncoder
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from keras.preprocessing.text import Tokenizer
from nltk.stem.lancaster import LancasterStemmer
import json 
import random
import re
import string
from nltk.stem.snowball import SnowballStemmer
from keras.models import Sequential,load_model
from keras.layers import Dense, LSTM, Bidirectional, Embedding, Dropout
from keras.callbacks import ModelCheckpoint
from keras.preprocessing.sequence import pad_sequences
import nltk
nltk.download('stopwords')
stopword = set(stopwords.words('english')) 

Using TensorFlow backend.
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\kb\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Loading data
### Loading it directly from GitHub. No need to save in local!

In [2]:
def data_loader(url):
    resp = requests.get(url)
    data = json.loads(resp.text)
    print("Data loaded successfully!")
    print("The dataset has following keys =",data.keys())
    return data

# Data Pre-processing

In [3]:

def data_preprocessing(data):
    # extracting test ,train and validation data from the .json file into dataframe
    full_test_data = pd.DataFrame(data['test'],columns=['text','label'])
    full_train_data = pd.DataFrame(data['train'],columns=['text','label'])
    full_val_data = pd.DataFrame(data['val'],columns=['text','label'])

    #identifying all labels in the dataset and storing it in a list.
    label_list=full_train_data['label'].unique().tolist()

    # Using random getting 20 random labels from the total labels

    k=0
    d={}
    while(k!=20):
        x = random.randint(0,len(label_list))
        if x in d:
            continue
        d[x]=x
        k+=1
    for i in d:
        selected.append(label_list[i-1])

    #Getting the train data with the selected labels
    train_20labels=pd.DataFrame()
    for i in selected:
        df=full_train_data[full_train_data['label']==i]
        frames=[train_20labels,df]
        train_20labels=pd.concat(frames)

    #Getting the test data with the selected labels
    test_20labels=pd.DataFrame()
    for i in selected:
        df=full_test_data[full_test_data['label']==i]
        frames=[test_20labels,df]
        test_20labels=pd.concat(frames)
    
    #Getting the val data with the selected labels
    val_20labels=pd.DataFrame()
    for i in selected:
        df=full_val_data[full_val_data['label']==i]
        frames=[val_20labels,df]
        val_20labels=pd.concat(frames)
    return train_20labels,test_20labels,val_20labels





## Data Cleanising

In [4]:
def data_cleansing():
    # Removing punctuations
    def punctuation_removal(text):
        no_punct=[words for words in text if words not in string.punctuation]
        words_wo_punct=''.join(no_punct)
        return words_wo_punct

    # Tokenize strings
    def tokenize(text):
        split=re.split("\W+",text) 
        return split

    # Remove stopwords
    def remove_stopwords(text):
        text=[word for word in text if word not in stopword]
        return text

    #Defining Stemmer
    def stemmer(text):
        s_stemmer = SnowballStemmer(language='english')
        for word in text:
            text=[s_stemmer.stem(word) for word in text]
        return text
    train_20labels['text']=train_20labels['text'].apply(lambda x: punctuation_removal(x))
    train_20labels['text']=train_20labels['text'].apply(lambda x: tokenize(x.lower()))
    train_20labels['text']=train_20labels['text'].apply(lambda x: remove_stopwords(x))
    #train_20labels['text']=train_20labels['text'].apply(lambda x: stemmer(x))

    test_20labels['text']=test_20labels['text'].apply(lambda x: punctuation_removal(x))
    test_20labels['text']=test_20labels['text'].apply(lambda x: tokenize(x.lower()))
    test_20labels['text']=test_20labels['text'].apply(lambda x: remove_stopwords(x))
    #test_20labels['text']=test_20labels['text'].apply(lambda x: stemmer(x))

    val_20labels['text']=val_20labels['text'].apply(lambda x: punctuation_removal(x))
    val_20labels['text']=val_20labels['text'].apply(lambda x: tokenize(x.lower()))
    val_20labels['text']=val_20labels['text'].apply(lambda x: remove_stopwords(x))
    #val_20labels['text']=val_20labels['text'].apply(lambda x: stemmer(x))
    train_20labels['label']=train_20labels['label'].str.replace('_','')
    test_20labels['label']=test_20labels['label'].str.replace('_','')
    val_20labels['label']=val_20labels['label'].str.replace('_','')

# Data Representation and feature selection

In [5]:
d={}
def data_representation():
    def changing1(t,x):
        a=[0 for i in range(20)]
        a[t.word_index.get(x)-1]=1
        d[t.word_index.get(x)-1]=x
        return a

    def changing(x):
        q=t.texts_to_sequences(x)
        r=[]
        if len(q)!=0:
            for i in q:
                r.append(i[0])
        c=max_length-len(r)
        z=[0 for i in range(c)]
        r=r+z
        return np.asarray(r)

    r=pd.DataFrame()
    frames=[train_20labels['text'],test_20labels['text'],val_20labels['text']]
    r['text']=pd.concat(frames)
    s=set()

    ##Input tokenizer for vectorizing features ie. text data for all test,train and val dataframe
    t = Tokenizer()
    t.fit_on_texts(r['text'])
    for i in range(len(r['text'])):
        for j in range(len(r['text'].iloc[i])):
            s.add(r['text'].iloc[i][j])

    vocab_size=len(s)
    max_length=int(train_20labels['text'].str.len().max())

    #applying the changes to train/test/val datarame
    train_20labels['matrix']= train_20labels['text'].apply(lambda x:changing(x))
    test_20labels['matrix']= test_20labels['text'].apply(lambda x: changing(x))
    val_20labels['matrix']= val_20labels['text'].apply(lambda x: changing(x))


    ##Output tokenizer for encoding labels

    ot=Tokenizer()
    ot.fit_on_texts(train_20labels['label'].values.tolist())


    #applying the changes to train/test/val datarame
    train_20labels['elabel']= train_20labels['label'].apply(lambda x: changing1(ot,x))
    test_20labels['elabel']= test_20labels['label'].apply(lambda x: changing1(ot,x))
    val_20labels['elabel']= val_20labels['label'].apply(lambda x: changing1(ot,x))
    train_20labels['elabel'] = np.array(train_20labels['elabel']).reshape(len(train_20labels['elabel']), 1)
    test_20labels['elabel'] = np.array(test_20labels['elabel']).reshape(len(test_20labels['elabel']), 1)
    val_20labels['elabel'] = np.array(val_20labels['elabel']).reshape(len(val_20labels['elabel']), 1)

    return vocab_size,max_length

In [6]:
def data_preperation():
    train_X=train_20labels['matrix']
    train_Y =train_20labels['elabel']
    val_X =val_20labels['matrix']
    val_Y =val_20labels['elabel']
    test_X=test_20labels['matrix']
    test_Y =test_20labels['elabel']

    # reshaping train_X and train_Y
    train_X=train_X.to_numpy()
    train_X=np.vstack(train_X)
    train_Y=train_Y.to_numpy()
    train_Y=np.vstack(train_Y)

    # reshaping val_X and val_Y
    val_X=val_X.to_numpy()
    val_X=np.vstack(val_X)
    val_Y=val_Y.to_numpy()
    val_Y=np.vstack(val_Y)

    # reshaping test_X and test_Y
    test_X=test_X.to_numpy()
    test_X=np.vstack(test_X)
    test_Y=test_Y.to_numpy()
    test_Y=np.vstack(test_Y)
    return train_X,test_X,val_X,train_Y,test_Y,val_Y

In [7]:
def LSTM_model(vocab_size, max_length,train_X,val_X,train_Y,val_Y):
    def create_model(vocab_size, max_length):
        model = Sequential()
        model.add(Embedding(vocab_size, 64, input_length = max_length, trainable = False))
        model.add(Bidirectional(LSTM(256)))
        model.add(Dense(32, activation = "relu"))
        model.add(Dropout(0.5))
        model.add(Dense(20, activation = "softmax"))
        return model

    model = create_model(vocab_size+1, max_length)
    model.compile(loss = "categorical_crossentropy", optimizer = "adam", metrics = ["accuracy"])
    model.summary()
    filename = 'model.h5'
    checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

    hist = model.fit(train_X, train_Y, epochs = 200, batch_size = 16, shuffle=True, validation_data = (val_X, val_Y), callbacks = [checkpoint])

In [8]:
def predict(test_X):
    def predictions():
        model = load_model("model.h5")
        pred=model.predict(test_X)
        return pred
    pred=predictions()
    def get_final_output(pred, classes):
      print(len(pred))  
      for i in range(len(pred)):
        predictions = pred[i]
        classes = np.array(classes)
        ids = np.argsort(-predictions)
        classes = classes[ids]
        predictions = -np.sort(-predictions)
        print("Testing for statement :", question[i])
        true=''
        for j in range(len(test_Y[i])):
            if test_Y[i][j]==1:
                true=d[j+1]
                break
        print('\n Results are:')
        for i in range(pred.shape[1]):
            print("%s has confidence = %s" % (classes[i], (predictions[i])))
        print('------------------------------------------------------------------------')
    p=list(selected)
    for i in range(len(selected)):
        p[i]=p[i].replace('_','')
    get_final_output(pred, p)


In [9]:
data=data_loader("https://raw.githubusercontent.com/clinc/oos-eval/master/data/data_full.json")
selected=[]
train_20labels,test_20labels,val_20labels = data_preprocessing(data)
finaldf=test_20labels
question=[]
for i in range(len(finaldf)):
    question.append(finaldf['text'].iloc[i])
print("\nRandomly Selected 20 labels from in-scope content are =\n", selected)
data_cleansing()
vocab_size,max_length=data_representation()
train_X,test_X,val_X,train_Y,test_Y,val_Y=data_preperation()
LSTM_model(vocab_size, max_length,train_X,val_X,train_Y,val_Y)


Data loaded successfully!
The dataset has following keys = dict_keys(['oos_val', 'val', 'train', 'oos_test', 'test', 'oos_train'])

Randomly Selected 20 labels from in-scope content are =
 ['what_song', 'bill_due', 'min_payment', 'new_card', 'oil_change_how', 'pto_used', 'change_accent', 'cook_time', 'pin_change', 'travel_alert', 'what_are_your_hobbies', 'are_you_a_bot', 'recipe', 'improve_credit_score', 'rewards_balance', 'next_song', 'cancel', 'timezone', 'where_are_you_from', 'change_volume']
Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 13, 64)            85632     
_________________________________________________________________
bidirectional_1 (Bidirection (None, 512)               657408    
_________________________________________________________________
dense_1 (Dense)              (None, 32)                16416     
_______________


Epoch 00031: val_loss did not improve from 0.66912
Epoch 32/200
2000/2000 [==============================] - 9s 5ms/step - loss: 0.4423 - accuracy: 0.8575 - val_loss: 0.8198 - val_accuracy: 0.7800

Epoch 00032: val_loss did not improve from 0.66912
Epoch 33/200
2000/2000 [==============================] - 9s 4ms/step - loss: 0.4530 - accuracy: 0.8440 - val_loss: 0.6852 - val_accuracy: 0.8175

Epoch 00033: val_loss did not improve from 0.66912
Epoch 34/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.4105 - accuracy: 0.8630 - val_loss: 0.8885 - val_accuracy: 0.7500

Epoch 00034: val_loss did not improve from 0.66912
Epoch 35/200
2000/2000 [==============================] - 9s 5ms/step - loss: 0.4986 - accuracy: 0.8385 - val_loss: 0.9139 - val_accuracy: 0.7775

Epoch 00035: val_loss did not improve from 0.66912
Epoch 36/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.4403 - accuracy: 0.8595 - val_loss: 0.7023 - val_accuracy: 0.8125

Epoch 000

2000/2000 [==============================] - 8s 4ms/step - loss: 0.1888 - accuracy: 0.9360 - val_loss: 0.7728 - val_accuracy: 0.8425

Epoch 00073: val_loss did not improve from 0.66912
Epoch 74/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.1748 - accuracy: 0.9400 - val_loss: 0.8889 - val_accuracy: 0.8350

Epoch 00074: val_loss did not improve from 0.66912
Epoch 75/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.1572 - accuracy: 0.9495 - val_loss: 0.9097 - val_accuracy: 0.8425

Epoch 00075: val_loss did not improve from 0.66912
Epoch 76/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.2243 - accuracy: 0.9335 - val_loss: 1.1270 - val_accuracy: 0.7925

Epoch 00076: val_loss did not improve from 0.66912
Epoch 77/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.3013 - accuracy: 0.9075 - val_loss: 0.8573 - val_accuracy: 0.8325

Epoch 00077: val_loss did not improve from 0.66912
Epoch 78/200
2000/2000 

2000/2000 [==============================] - 8s 4ms/step - loss: 0.1087 - accuracy: 0.9615 - val_loss: 0.9730 - val_accuracy: 0.8350

Epoch 00114: val_loss did not improve from 0.66912
Epoch 115/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.1339 - accuracy: 0.9555 - val_loss: 1.1562 - val_accuracy: 0.8100

Epoch 00115: val_loss did not improve from 0.66912
Epoch 116/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.1544 - accuracy: 0.9475 - val_loss: 1.1055 - val_accuracy: 0.8425

Epoch 00116: val_loss did not improve from 0.66912
Epoch 117/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.1142 - accuracy: 0.9620 - val_loss: 0.9265 - val_accuracy: 0.8350

Epoch 00117: val_loss did not improve from 0.66912
Epoch 118/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.1295 - accuracy: 0.9590 - val_loss: 1.0493 - val_accuracy: 0.8350

Epoch 00118: val_loss did not improve from 0.66912
Epoch 119/200
2000/

2000/2000 [==============================] - 8s 4ms/step - loss: 0.0974 - accuracy: 0.9605 - val_loss: 1.1188 - val_accuracy: 0.8375

Epoch 00155: val_loss did not improve from 0.66912
Epoch 156/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.0934 - accuracy: 0.9680 - val_loss: 1.1399 - val_accuracy: 0.8425

Epoch 00156: val_loss did not improve from 0.66912
Epoch 157/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.0929 - accuracy: 0.9705 - val_loss: 0.9996 - val_accuracy: 0.8525

Epoch 00157: val_loss did not improve from 0.66912
Epoch 158/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.0813 - accuracy: 0.9710 - val_loss: 1.3063 - val_accuracy: 0.8450

Epoch 00158: val_loss did not improve from 0.66912
Epoch 159/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.0863 - accuracy: 0.9700 - val_loss: 1.2784 - val_accuracy: 0.8475

Epoch 00159: val_loss did not improve from 0.66912
Epoch 160/200
2000/

2000/2000 [==============================] - 11s 5ms/step - loss: 0.0729 - accuracy: 0.9760 - val_loss: 1.5284 - val_accuracy: 0.8300

Epoch 00196: val_loss did not improve from 0.66912
Epoch 197/200
2000/2000 [==============================] - 8s 4ms/step - loss: 0.1066 - accuracy: 0.9665 - val_loss: 1.3443 - val_accuracy: 0.8050

Epoch 00197: val_loss did not improve from 0.66912
Epoch 198/200
2000/2000 [==============================] - 10s 5ms/step - loss: 0.0826 - accuracy: 0.9770 - val_loss: 1.0730 - val_accuracy: 0.8475

Epoch 00198: val_loss did not improve from 0.66912
Epoch 199/200
2000/2000 [==============================] - 10s 5ms/step - loss: 0.2220 - accuracy: 0.9495 - val_loss: 0.8471 - val_accuracy: 0.8425

Epoch 00199: val_loss did not improve from 0.66912
Epoch 200/200
2000/2000 [==============================] - 9s 5ms/step - loss: 0.1114 - accuracy: 0.9620 - val_loss: 1.0449 - val_accuracy: 0.8350

Epoch 00200: val_loss did not improve from 0.66912


In [10]:
predict(test_X)

600
Testing for statement : i'd like to know what song this is

 Results are:
whatsong has confidence = 0.806169
nextsong has confidence = 0.19376695
cancel has confidence = 3.6549194e-05
areyouabot has confidence = 1.649176e-05
whereareyoufrom has confidence = 9.901168e-06
travelalert has confidence = 5.838595e-07
timezone has confidence = 1.8465894e-07
recipe has confidence = 1.6298023e-07
newcard has confidence = 1.114715e-07
changeaccent has confidence = 1.0841356e-07
billdue has confidence = 4.760512e-09
changevolume has confidence = 4.7182737e-11
rewardsbalance has confidence = 3.1274643e-11
cooktime has confidence = 8.292536e-12
oilchangehow has confidence = 3.1577245e-13
ptoused has confidence = 2.5333232e-13
pinchange has confidence = 1.8328463e-13
whatareyourhobbies has confidence = 3.5634984e-14
minpayment has confidence = 2.2532733e-14
improvecreditscore has confidence = 2.52831e-15
------------------------------------------------------------------------
Testing for stateme

Testing for statement : what is my minimum payment and when is it due

 Results are:
newcard has confidence = 0.89368975
whatsong has confidence = 0.10623901
cooktime has confidence = 4.451832e-05
whereareyoufrom has confidence = 2.47517e-05
travelalert has confidence = 1.432071e-06
changevolume has confidence = 4.4717285e-07
billdue has confidence = 2.4758663e-07
improvecreditscore has confidence = 7.683786e-10
oilchangehow has confidence = 1.3926144e-10
whatareyourhobbies has confidence = 2.2159788e-11
rewardsbalance has confidence = 1.8758403e-11
cancel has confidence = 4.4562715e-14
pinchange has confidence = 2.1802303e-14
nextsong has confidence = 1.740932e-14
recipe has confidence = 2.6232343e-16
areyouabot has confidence = 1.5531006e-16
minpayment has confidence = 3.7922254e-17
timezone has confidence = 1.5502528e-17
ptoused has confidence = 1.1745968e-18
changeaccent has confidence = 6.337583e-20
------------------------------------------------------------------------
Testing f

changevolume has confidence = 3.9326483e-11
whatsong has confidence = 3.367926e-11
pinchange has confidence = 2.817461e-11
nextsong has confidence = 1.0055014e-11
cooktime has confidence = 5.655153e-12
improvecreditscore has confidence = 1.3646919e-12
whereareyoufrom has confidence = 1.929128e-13
whatareyourhobbies has confidence = 1.1420009e-13
oilchangehow has confidence = 7.9977935e-14
changeaccent has confidence = 4.5481418e-17
rewardsbalance has confidence = 2.207237e-18
newcard has confidence = 1.6125406e-18
areyouabot has confidence = 2.5453024e-21
recipe has confidence = 1.9538079e-21
ptoused has confidence = 1.83459e-24
cancel has confidence = 3.8358358e-27
minpayment has confidence = 9.8357284e-29
------------------------------------------------------------------------
Testing for statement : how do i do an oil change

 Results are:
whatsong has confidence = 0.97980493
nextsong has confidence = 0.016041467
whereareyoufrom has confidence = 0.0022966578
cooktime has confidence 

changeaccent has confidence = 7.151626e-05
cancel has confidence = 4.3655727e-05
improvecreditscore has confidence = 2.2422635e-05
ptoused has confidence = 1.3935495e-05
whatareyourhobbies has confidence = 1.046564e-05
pinchange has confidence = 8.409964e-06
areyouabot has confidence = 3.1045759e-06
oilchangehow has confidence = 3.838353e-07
timezone has confidence = 3.7632046e-07
whereareyoufrom has confidence = 3.7081347e-07
travelalert has confidence = 7.2240915e-08
recipe has confidence = 1.1075439e-08
rewardsbalance has confidence = 8.952876e-09
------------------------------------------------------------------------
Testing for statement : how much of my pto is gone

 Results are:
minpayment has confidence = 0.75385
whatsong has confidence = 0.2192526
pinchange has confidence = 0.009499758
rewardsbalance has confidence = 0.009090521
oilchangehow has confidence = 0.0039711082
changeaccent has confidence = 0.0015586042
improvecreditscore has confidence = 0.001089843
whereareyoufrom

Testing for statement : how many minutes should i set an alarm for this bake

 Results are:
timezone has confidence = 0.64314383
travelalert has confidence = 0.32527965
nextsong has confidence = 0.008361616
recipe has confidence = 0.0050877687
whatareyourhobbies has confidence = 0.004379072
whereareyoufrom has confidence = 0.0043356433
rewardsbalance has confidence = 0.0034410893
oilchangehow has confidence = 0.002323452
pinchange has confidence = 0.0011807646
whatsong has confidence = 0.00093916204
newcard has confidence = 0.0008029842
billdue has confidence = 0.0002211469
areyouabot has confidence = 0.0001858671
improvecreditscore has confidence = 0.00018571968
minpayment has confidence = 7.870754e-05
changevolume has confidence = 2.2164728e-05
changeaccent has confidence = 2.1328626e-05
cancel has confidence = 6.0402176e-06
ptoused has confidence = 2.6423618e-06
cooktime has confidence = 1.4503796e-06
------------------------------------------------------------------------
Testing f

...

newcard has confidence = 2.3503413e-10
changevolume has confidence = 1.6743264e-10
pinchange has confidence = 2.465109e-11
whatareyourhobbies has confidence = 2.4383549e-12
ptoused has confidence = 3.410042e-13
rewardsbalance has confidence = 1.13863564e-14
timezone has confidence = 6.466008e-15
cancel has confidence = 8.061975e-16
oilchangehow has confidence = 2.020565e-16
billdue has confidence = 1.01078506e-16
recipe has confidence = 2.05152e-19
improvecreditscore has confidence = 1.9204279e-20
------------------------------------------------------------------------
Testing for statement : how can i up my credit score

 Results are:
pinchange has confidence = 0.41307455
rewardsbalance has confidence = 0.2659186
recipe has confidence = 0.17909303
oilchangehow has confidence = 0.104275756
changeaccent has confidence = 0.008398033
travelalert has confidence = 0.0077996054
improvecreditscore has confidence = 0.007368936
ptoused has confidence = 0.004178266
whatareyourhobbies has confide

Testing for statement : tell me my debit card's rewards balance

 Results are:
improvecreditscore has confidence = 0.8857528
areyouabot has confidence = 0.081364416
changevolume has confidence = 0.023950212
whereareyoufrom has confidence = 0.001976275
minpayment has confidence = 0.0018575215
whatareyourhobbies has confidence = 0.0018072139
changeaccent has confidence = 0.0016733077
timezone has confidence = 0.0013612104
recipe has confidence = 8.905849e-05
billdue has confidence = 5.6521043e-05
cooktime has confidence = 4.0700328e-05
newcard has confidence = 2.8860304e-05
ptoused has confidence = 2.6550922e-05
cancel has confidence = 5.9839676e-06
whatsong has confidence = 4.385126e-06
nextsong has confidence = 2.718674e-06
travelalert has confidence = 1.4954655e-06
rewardsbalance has confidence = 7.4268365e-07
pinchange has confidence = 8.587993e-08
oilchangehow has confidence = 2.2977378e-09
------------------------------------------------------------------------
Testing for statemen

billdue has confidence = 4.5255343e-05
rewardsbalance has confidence = 1.7815364e-05
changevolume has confidence = 5.9888757e-06
changeaccent has confidence = 1.297481e-06
pinchange has confidence = 7.247953e-07
timezone has confidence = 1.2436558e-07
nextsong has confidence = 3.5682497e-09
cancel has confidence = 3.496518e-09
ptoused has confidence = 2.9926077e-09
minpayment has confidence = 7.833771e-10
newcard has confidence = 5.730096e-10
improvecreditscore has confidence = 1.1932999e-10
whatsong has confidence = 2.5012815e-12
oilchangehow has confidence = 7.166606e-13
cooktime has confidence = 2.3787534e-13
recipe has confidence = 1.2374102e-15
whatareyourhobbies has confidence = 1.1385426e-16
------------------------------------------------------------------------
Testing for statement : i want you to play the next song

 Results are:
whatsong has confidence = 0.99969554
whereareyoufrom has confidence = 0.00030443328
oilchangehow has confidence = 2.1552916e-11
changeaccent has co

minpayment has confidence = 0.0031325724
billdue has confidence = 0.0020445404
areyouabot has confidence = 0.0019469142
changeaccent has confidence = 0.0018003825
pinchange has confidence = 0.0016906634
whatsong has confidence = 0.000989344
cooktime has confidence = 0.0007938056
newcard has confidence = 0.0007058556
travelalert has confidence = 0.0005543062
cancel has confidence = 0.00047063056
improvecreditscore has confidence = 0.00025933643
oilchangehow has confidence = 0.00014113491
timezone has confidence = 8.9801375e-05
whereareyoufrom has confidence = 4.4763376e-05
whatareyourhobbies has confidence = 4.4688008e-05
recipe has confidence = 4.0179624e-05
ptoused has confidence = 5.498084e-06
------------------------------------------------------------------------
Testing for statement : where are you coming from

 Results are:
recipe has confidence = 0.53748465
whereareyoufrom has confidence = 0.27097082
newcard has confidence = 0.15593562
travelalert has confidence = 0.015730599
r

KeyError: 20